In [75]:
import requests
import codecs
import csv
from bs4 import BeautifulSoup 
import json
import io
import gzip
from html.parser import HTMLParser

In [76]:
get_requests1 = requests.get('http://index.commoncrawl.org/CC-MAIN-2019-09-index?url=http%3A%2F%2Fhttps://www.nfl.com/&output=json')
get_requests2 = requests.get('http://index.commoncrawl.org/CC-MAIN-2019-04-index?url=http%3A%2F%2Fhttps://www.nfl.com/&output=json')
requestsArray = []
requestsArray.append(get_requests1)
requestsArray.append(get_requests2)

In [77]:
pageList = []
for r in requestsArray:
    if r.status_code == 200:
        pages = r.content.splitlines()
        for i in pages:
            pageList.append(json.loads(i))
            
#pages = [json.loads(x) for x in get_requests.content.decode('utf8').strip().split('\n')]
#page = pages[2]

{'urlkey': 'com,nfl)/', 'timestamp': '20190215230006', 'url': 'http://www.nfl.com/', 'filename': 'crawl-data/CC-MAIN-2019-09/segments/1550247479627.17/crawldiagnostics/CC-MAIN-20190215224408-20190216010408-00048.warc.gz', 'mime-detected': 'application/octet-stream', 'offset': '10629915', 'digest': '3I42H3S6NNFQ2MSVX7XZKYAYSCX5QBYJ', 'status': '301', 'length': '556', 'mime': 'unk'}
{'urlkey': 'com,nfl)/', 'timestamp': '20190215230006', 'url': 'https://www.nfl.com/', 'filename': 'crawl-data/CC-MAIN-2019-09/segments/1550247479627.17/warc/CC-MAIN-20190215224408-20190216010408-00271.warc.gz', 'mime-detected': 'text/html', 'offset': '907412876', 'digest': 'USICROEUUY5F7WMUXFW43C4NXKJHHZ4S', 'languages': 'eng', 'status': '200', 'length': '23676', 'mime': 'text/html', 'charset': 'UTF-8'}
{'urlkey': 'com,nfl)/', 'timestamp': '20190216020939', 'url': 'https://www.nfl.com/', 'filename': 'crawl-data/CC-MAIN-2019-09/segments/1550247479729.27/robotstxt/CC-MAIN-20190216004609-20190216030609-00271.war

In [68]:
page = pageList[1]
print(page)

{'urlkey': 'com,nfl)/', 'timestamp': '20190215230006', 'url': 'https://www.nfl.com/', 'filename': 'crawl-data/CC-MAIN-2019-09/segments/1550247479627.17/warc/CC-MAIN-20190215224408-20190216010408-00271.warc.gz', 'mime-detected': 'text/html', 'offset': '907412876', 'digest': 'USICROEUUY5F7WMUXFW43C4NXKJHHZ4S', 'languages': 'eng', 'status': '200', 'length': '23676', 'mime': 'text/html', 'charset': 'UTF-8'}


In [69]:
offset, length = int(page['offset']), int(page['length'])
offset_end = offset + length - 1
print(offset, length, offset_end)

#prefix = 'https://aws-publicdatasets.s3.amazonaws.com/'
prefix = 'https://commoncrawl.s3.amazonaws.com/'

response = requests.get(prefix + page['filename'], headers={'Range': 'bytes={}-{}'.format(offset, offset_end)})
print(response)

907412876 23676 907436551
<Response [206]>


In [70]:
raw_data = io.BytesIO(response.content)
print(raw_data)
f = gzip.GzipFile(fileobj=raw_data)
print(f)

<gzip _io.BytesIO object at 0x1112bc0a0 0x111489eb8>


In [71]:
data = f.read()
#print(data)
#resp = ""
#if len(data):
#    try:
#        resp = data.decode('utf8').strip().split('\r\n\r\n', 2)
#    except:
#        pass
#print(resp)

In [72]:
#doc = parse(io.BytesIO(data)).getroot()
doc = BeautifulSoup(io.BytesIO(data), 'lxml')
#print(doc)
all_a = doc.find_all('a')
hrefList = []
if all_a:
    for l in all_a:
        #print(l)
        href = l.attrs.get('href')
        
        if href is not None:
            if href.startswith('http'):
                hrefList.append(href)
        
#print(hrefList)

#print(all_a)
#print(doc)
#for div in doc.cssselect('a'):
#    print('%s: %s' % (div.text_content(), div.get('href')))

In [73]:
with codecs.open('links.csv', 'wb', encoding = 'utf8') as output:
    f = ['url']
    write_csv = csv.DictWriter(output, fieldnames = f)
    write_csv.writeheader()
    
    for h in hrefList:
        write_csv.writerow({'url':h})

In [74]:
#hrefList[1]
for h in hrefList:
    #print(hrefList[1])
    print(h)
#p = requests.get(hrefList[1])
#print(p.status_code)
#print(p.text)
'''pageListFinal = []
if p.status_code == 200:
    pagesFinal = p.content
    print(pagesFinal)
    for i in pagesFinal:
        pageListFinal.append(json.loads(i))'''
        
#print(pageListFinal[0])

https://www.nfl.com
https://www.nfl.com/news
https://www.nfl.com/schedules
https://www.nfl.com/gamepass?icampaign=nfl-nav-gamepass
https://www.nfl.com/draft
https://www.nfl.com/freeagency


'pageListFinal = []\nif p.status_code == 200:\n    pagesFinal = p.content\n    print(pagesFinal)\n    for i in pagesFinal:\n        pageListFinal.append(json.loads(i))'